In [ ]:
%reload_ext autoreload
%autoreload 2 
    
import pandas as pd
import numpy as np
import keras_tuner as kt
from keras_tuner import RandomSearch, HyperModel, Hyperband, HyperParameters
from tensorflow.keras.callbacks import EarlyStopping

from DataEncoder import encode_pad_event, encode_pad_sequence, encode_y, encode_textual_event
from DurationEmbedding import duration_embedding_layer
from FeatureEmbedding import feature_embedding_layer
#from DurationEmbeddingLSTM import DurationEmbeddingLSTMModel, print_best_hp_duration
from TextFeatureDurationEmbeddingLSTMIm import TextFeatureDurationEmbeddingLSTMModel, print_best_hp_duration_text

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score

In [ ]:
event = pd.read_csv("../input/Event_Feature_pro.csv")
sequence = pd.read_csv("../input/Sequence_Feature_pro.csv")
event_duration_embedding = pd.read_csv("../output/Event_Duration_Embedding_pro.csv")
event_feature_embedding = pd.read_csv("../output/Event_feature_Embedding_pro.csv")

In [ ]:
y_col = sequence.result
y_encode = encode_y(y_col)

In [ ]:
cat_col_event = ['Resource', 'outcome', "stopcode"]
num_col_event = ['net_promotor_score', 'creditscore', 'rate_charged']
case_index = 'Case ID'

event_encode = encode_pad_event(event, cat_col_event, num_col_event, case_index, cat_mask = True, num_mask = True, eos = False)

In [ ]:
cat_col_seq = ['plan']
num_col_seq = ['age', 'coverage_numeric', 'length_of_stay']
sequence_encode = encode_pad_sequence(sequence, cat_col_seq, num_col_seq,  cat_mask = False, num_mask = False)

In [ ]:
event_duration_embedding_names = event_duration_embedding.columns.to_list()
embedding_duration_cols_names = event_duration_embedding_names[7:]
duration_embedding_encode = duration_embedding_layer(event_duration_embedding, embedding_duration_cols_names, case_index, eos = False)

In [ ]:
event_feature_embedding_names = event_feature_embedding.columns.to_list()
embedding_feature_cols_names = event_feature_embedding_names[5:]
feature_embedding_encode = feature_embedding_layer(event_feature_embedding, embedding_feature_cols_names, case_index, eos = False)

In [ ]:
verb_encode, verb_size = encode_textual_event(event, 'Activity_verb', case_index, eos = False)
dec_encode, dec_size = encode_textual_event(event, 'Activity_Dec', case_index, eos = False)

In [ ]:
# Define the input shape (sequence_length, number_of_features)
event_input_shape = (event_encode.shape[1], event_encode.shape[2])

# Define the number of classes 
num_classes = y_encode.shape[1]

# number of sequence features
num_sequence_features = sequence_encode.shape[1]

duration_embedding_shape = (duration_embedding_encode.shape[1], duration_embedding_encode.shape[2])
feature_embedding_shape = (feature_embedding_encode.shape[1], feature_embedding_encode.shape[2])

# make sure the event_encode.shape[1] = duration(feature)_embedding_encode.shape[1] = verb(dec)encode.shape[1] 
sequence_size = event_encode.shape[1]

In [ ]:
# Split indices for train and test
train_indices, test_indices = train_test_split(np.arange(len(y_encode)), test_size=0.2, stratify=y_encode, random_state=42)

# Split the data
train_event_features = event_encode[train_indices]
test_event_features = event_encode[test_indices]

train_sequence_features = sequence_encode[train_indices]
test_sequence_features = sequence_encode[test_indices]

train_y = y_encode[train_indices]
test_y = y_encode[test_indices]

train_duration_embedding = duration_embedding_encode[train_indices]
test_duration_embedding = duration_embedding_encode[test_indices]

train_feature_embedding = feature_embedding_encode[train_indices]
test_feature_embedding = feature_embedding_encode[test_indices]

train_verb_encode = verb_encode[train_indices]
train_dec_encode = dec_encode[train_indices]

test_verb_encode = verb_encode[test_indices]
test_dec_encode = dec_encode[test_indices]

In [ ]:
# Include batch size as a hyperparameter to be tuned
hp_b = HyperParameters()
batch_size = hp_b.Choice('batch_size', values=[16, 32, 64, 128])

In [ ]:
#Initialize the hypermodel
hypermodel = TextFeatureDurationEmbeddingLSTMModel(sequence_size = sequence_size, 
                                                   verb_size = verb_size, 
                                                   dec_size = dec_size,
                                                   event_input_shape = event_input_shape,
                                                   num_sequence_features = num_sequence_features,
                                                   feature_embedding_shape = feature_embedding_shape,
                                                   duration_embedding_shape = duration_embedding_shape,
                                                   num_classes=num_classes)

early_stopping = EarlyStopping(monitor='val_f1_score',  
                               mode='max',
                               patience=5, 
                               restore_best_weights=True)

tuner_band = Hyperband(hypermodel,
                       objective=kt.Objective("val_f1_score", direction="max"),
                       max_epochs=200,
                       factor=3,
                       directory='hparam_tuning',
                       overwrite=False, #if resume tuner, keep overwrite = False
                       project_name='classfication_feature_text_hyperband_im',
                       hyperparameters=hp_b)

# Use the manually split data in the search
tuner_band.search(x=[train_verb_encode, train_dec_encode, train_event_features, train_feature_embedding, train_duration_embedding, train_sequence_features], y=train_y, 
                  validation_data=([test_verb_encode, test_dec_encode, test_event_features, test_feature_embedding, test_duration_embedding, test_sequence_features], test_y), 
                  epochs=200, callbacks=[early_stopping])

# Get the optimal hyperparameters
best_hps_band = tuner_band.get_best_hyperparameters(num_trials=1)[0]

# Get the best model
best_model_band = tuner_band.get_best_models(num_models=1)[0]

In [ ]:
# Get predictions
y_pred_probs = best_model_band.predict([test_verb_encode, test_dec_encode, test_event_features, test_feature_embedding, test_duration_embedding, test_sequence_features])
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert to class indices
y_true = np.argmax(test_y, axis=1)

# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Compute Classification Report
class_report = classification_report(y_true, y_pred, digits=4)
print("Classification Report:")
print(class_report)

In [ ]:
# alternatively, build the model with the optimal hyperparameters 
# best_model_band  = band_tuner.hypermodel.build(best_hps_band)
history_band = best_model_band.fit(x=[train_verb_encode, train_dec_encode, train_event_features, train_feature_embedding, train_duration_embedding, train_sequence_features], y=train_y, 
                                   validation_data=([test_verb_encode, test_dec_encode, test_event_features, test_feature_embedding, test_duration_embedding, test_sequence_features], test_y), 
                                   epochs=5, batch_size=best_hps_band.get('batch_size'), callbacks=[early_stopping])

val_acc_per_epoch_band = history_band.history['val_f1_score']
best_epoch_band = val_acc_per_epoch_band.index(max(val_acc_per_epoch_band)) + 1
print('Best epoch: %d' % (best_epoch_band,))
print('Best F1 Score: ', max(val_acc_per_epoch_band))

print_best_hp_duration_text(best_hps_band)